Setup

In [1]:
from itertools import product 
from pathlib import Path

import matplotlib.pyplot as plt
import torch

from helpers import models, data, plot

In [2]:
splits = ["train", "val"]
levels = ["gen", "det", "det_bkg"]
q_sq_veto = "resonances"

dset_name = "sets_unbinned"

def make_model_name(level, num_signal): return f"deep_sets_{level}_{num_signal}"

dc9_new_phys = -0.82

num_signal_per_set = [8_000, 16_000, 32_000]
num_sets_per_label = {8_000 : 400, 16_000: 200, 32_000 : 100} 
num_sets_sensitivity = 2_000
bkg_signal_ratio = 0.96
charge_bkg_fraction = 0.33

device = models.select_device()
loss_fn = torch.nn.MSELoss()
lr = 3e-4
lr_reduce_factor = 0.8
lr_reduce_patience = 5
batch_sizes = {8_000 : 128, 16_000 : 64, 32_000 : 32}
epochs = 100
epochs_checkpoint = 1

Device:  cuda


Save standard scaling constants

In [4]:
split = "train"

for level, num_signal in product(("det_bkg",), num_signal_per_set):

        sets_features, sets_labels = data.make_sets(
            level,
            split,
            q_sq_veto,
            num_signal,
            num_sets_per_label[num_signal],
            bkg_signal_ratio=bkg_signal_ratio,
            charge_bkg_fraction=charge_bkg_fraction
        )

        std_scale_mean = torch.mean(sets_features, dim=(0,1))
        std_scale_std = torch.std(sets_features, dim=(0,1))

        data.save_dset_file(std_scale_mean, dset_name, level, split, "mean", num_signal_per_set=num_signal)
        data.save_dset_file(std_scale_std, dset_name, level, split, "std", num_signal_per_set=num_signal)

        print(f"finished: {level}, {num_signal}")


finished: det_bkg, 8000
finished: det_bkg, 16000
finished: det_bkg, 32000


Dataset creation

In [5]:
for level, num_signal, split in product(levels, num_signal_per_set, splits): 

    sets_features, sets_labels = data.make_sets(
        level,
        split,
        q_sq_veto,
        num_signal,
        num_sets_per_label[num_signal],
        bkg_signal_ratio=bkg_signal_ratio,
        charge_bkg_fraction=charge_bkg_fraction
    )

    sets_features = data.apply_std_scale(sets_features, dset_name, level, num_signal_per_set=num_signal)

    data.save_dset_file(sets_features, dset_name, level, split, "features", num_signal_per_set=num_signal)
    data.save_dset_file(sets_labels, dset_name, level, split, "labels", num_signal_per_set=num_signal)

    print(f"finished: {level}, {num_signal}, {split}")


finished: gen, 8000
finished: gen, 8000
finished: gen, 16000
finished: gen, 16000
finished: gen, 32000
finished: gen, 32000
finished: det, 8000
finished: det, 8000
finished: det, 16000
finished: det, 16000
finished: det, 32000
finished: det, 32000
finished: det_bkg, 8000
finished: det_bkg, 8000
finished: det_bkg, 16000
finished: det_bkg, 16000
finished: det_bkg, 32000
finished: det_bkg, 32000


Sensitivity datasets

In [ ]:
split = "val"

for level, num_signal in product(levels, num_signal_per_set):

    sets_features, sets_labels = data.make_sets(
        level,
        split,
        num_signal,
        num_sets_sensitivity,
        label_subset=[dc9_new_phys],
        bkg_signal_ratio=bkg_signal_ratio,
        charge_bkg_fraction=charge_bkg_fraction
    )

    sets_features = data.apply_std_scale(sets_features, dset_name, level, num_signal_per_set=num_signal)

    helpers.data.save_dset_file(sets_features, dset_name, level, split, "sens_features", num_signal_per_set=num_signal)
    helpers.data.save_dset_file(sets_labels, dset_name, level, split, "sens_labels", num_signal_per_set=num_signal)

Train models

In [6]:
for level, num_signal in product(levels, num_signal_per_set):

    model = models.Deep_Sets_Model()

    model_name = make_model_name(level, num_signal)

    dataset_train = data.Dataset(dset_name, level, "train", num_signal_per_set=num_signal)
    dataset_val = data.Dataset(dset_name, level, "val", num_signal_per_set=num_signal)
    
    models.train(
        model,
        model_name,
        loss_fn,
        dataset_train,
        dataset_val,
        device,
        lr,
        lr_reduce_factor,
        lr_reduce_patience,
        batch_sizes[num_signal],
        batch_sizes[num_signal],
        epochs,
        epochs_checkpoint
    )

    print(f"finished: {level}, {num_signal}")



Epoch 0 complete:
    Train loss: 0.8513104915618896
    Val loss: 0.8246209025382996

Learning rate: [0.0003]
Peak GPU memory usage:  1.14 GB

Epoch 1 complete:
    Train loss: 0.8224759697914124
    Val loss: 0.8141427636146545

Learning rate: [0.0003]
Peak GPU memory usage:  1.14 GB

Epoch 2 complete:
    Train loss: 0.6249987483024597
    Val loss: 0.19346164166927338

Learning rate: [0.0003]
Peak GPU memory usage:  1.14 GB

Epoch 3 complete:
    Train loss: 0.08777537196874619
    Val loss: 0.04701065272092819

Learning rate: [0.0003]
Peak GPU memory usage:  1.14 GB

Epoch 4 complete:
    Train loss: 0.04012717306613922
    Val loss: 0.040444936603307724

Learning rate: [0.0003]
Peak GPU memory usage:  1.14 GB

Epoch 5 complete:
    Train loss: 0.038309190422296524
    Val loss: 0.03710710629820824

Learning rate: [0.0003]
Peak GPU memory usage:  1.14 GB

Epoch 6 complete:
    Train loss: 0.039631832391023636
    Val loss: 0.042449597269296646

Learning rate: [0.0003]
Peak GPU me

Evaluate models

Linearity and error

In [7]:
for level, num_signal in product(levels, num_signal_per_set):

    model_name = make_model_name(level, num_signal)
    model = models.Deep_Sets_Model()
    model.load_state_dict(models.open_model_state_dict(model_name))
    model.to(device)
    model.eval()
    
    dataset_val = data.Dataset(dset_name, level, "val", num_signal_per_set=num_signal)
    
    preds = models.predict_values_set_model(model, dataset_val.features, device)

    results_lin = models.run_linearity_test(preds, dataset_val.labels)
    results_err = models.run_error_test(preds, dataset_val.labels)

    models.save_test_result(results_lin, "lin", num_signal, model_name)
    models.save_test_result(results_err, "err", num_signal, model_name)

    print(f"finished: {level}, {num_signal}")

finished: gen, 8000
finished: gen, 16000
finished: gen, 32000
finished: det, 8000
finished: det, 16000
finished: det, 32000
finished: det_bkg, 8000
finished: det_bkg, 16000
finished: det_bkg, 32000


Sensitivity

In [ ]:
for level, num_signal in product(levels, num_signal_per_set):

    model_name = make_model_name(level, num_signal)
    model = models.Deep_Sets_Model()
    model.load_state_dict(models.open_model_state_dict(model_name))

    dataset_val_sens = data.Dataset(dset_name, level, "val", num_signal_per_set=num_signal, sensitivity=True)

    preds = models.predict_values_set_model(model, dataset_val_sens.features, device)

    results_sens = models.run_sensitivity_test(preds, dataset_val_sens.labels)

    models.save_test_result(results_sens, "sens", num_signal, model_name)

Plot results

Linearity

In [8]:
fig, axs = plt.subplots(3, 3, sharex=True, sharey=True, layout="compressed")

fancy_level_names = {
    "gen": "Generator", 
    "det" : "Detector", 
    "det_bkg" : "Detector and Bkg."
}

for (level, num_signal), ax in zip(product(levels, num_signal_per_set), axs.flat):
    
    model_name = make_model_name(level, num_signal)

    result = models.open_test_result("lin", num_signal, model_name)

    plot.plot_linearity(result, ax=ax)

    ax.set_title(
        f"Level: {fancy_level_names[level]}"
        f"\nEvents/set: {num_signal}"
        "\n" + r"Sets/$\delta C_9$: " + f"{num_sets_per_label[num_signal]}", 
        loc="left"
    )

axs.flat[0].legend()
fig.suptitle(f"Deep Sets\n", x=0.02, horizontalalignment="left")
fig.supxlabel(r"Actual $\delta C_9$", fontsize=11, x=0.56, y=-0.06)
fig.supylabel(r"Predicted $\delta C_9$", fontsize=11, y=0.45)

plt.savefig(Path("plots").joinpath("deep_sets_grid_lin.png"), bbox_inches="tight")
plt.close()